In [129]:
!pip install scikit-learn
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from sklearn import model_selection, datasets, linear_model, metrics

### Ансамбль методов для классификации

### Стекинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html

#### Обработка датасета

# Классификация

In [75]:
credit_df = pd.read_csv('../data/Credit_risk_preprocessed.csv')


In [76]:
credit_df["default"].unique()
x = credit_df.iloc[:,:-1].values
y = credit_df.iloc[:,4].values
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(x, y, test_size = 0.3, shuffle=False)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_cls)

X_train_cls = scaler.transform(X_train_cls)
X_test_cls = scaler.transform(X_test_cls)

### Стекинг без подбора гиперпараметров для него и базовых алгоритмов

In [36]:
estimators = [('lr', LogisticRegression()), ('dt', DecisionTreeClassifier())]
modelClf1 = StackingClassifier(estimators=estimators)

modelClf1.fit(X_train_cls, y_train_cls)
print(modelClf1.score(X_test_cls, y_test_cls))

0.975


#### Подбор гиперпараметров для базового алгоритма дерева решений

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn import tree
parameters = {'max_depth':[1, 2, 3, 4, 5], 'min_samples_leaf': range (1, 8) ,'min_samples_split': range (2,10,2)}
dtc = tree.DecisionTreeClassifier()
grid1_ = GridSearchCV(dtc, parameters)

In [38]:
grid1_.fit(X_train_cls, y_train_cls)
print(grid1_.best_params_)
print(grid1_.best_estimator_)

{'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 2}
DecisionTreeClassifier(max_depth=5, min_samples_leaf=3)


In [39]:
tr1 = tree.DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 1, min_samples_split = 2)
tr1 = tr1.fit(X_train_cls, y_train_cls)
y_pred1 = tr1.predict(X_test_cls)
print(tr1.score(X_test_cls, y_test_cls))

0.9416666666666667


#### Подбор гиперпараметров для базового алгоритма логистической регрессии

In [70]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import tree
parameters = {'penalty': ('l1', 'l2', 'elasticnet', 'none'),'solver' : ['liblinear', 'sag', 'saga'], 'C': np.logspace(-4, 4, 20) }
lr = LogisticRegression()
grid2_ = GridSearchCV(lr, parameters)
grid2_.fit(X_train_cls, y_train_cls)

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                         'penalty': ('l1', 'l2', 'elasticnet', 'none'),
                         'solver': ['liblinear', 'sag', 'saga']})

In [71]:
print(grid2_.best_params_)
print(grid2_.best_estimator_)

{'C': 0.23357214690901212, 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=0.23357214690901212, solver='liblinear')


In [72]:
lr1 = LogisticRegression(C=1.623776739188721, penalty='l1', solver='liblinear')
lr1.fit(X_train_cls, y_train_cls)
y_pred = lr1.predict(X_test_cls)
print(lr1.score(X_test_cls, y_test_cls))

0.945


## Бэггинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

### Бэггинг без подбора гиперпараметров для него и базовых алгоритмов

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf3 = BaggingClassifier(base_estimator=LogisticRegression())

modelClf3.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Логистическая регрессия":',modelClf3.score(X_test, y_test))

Бэггинг с базовым алгоритмом "Логистическая регрессия": 0.9483333333333334


In [52]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf4 = BaggingClassifier(base_estimator=DecisionTreeClassifier())

modelClf4.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Дерево решений":',modelClf4.score(X_test, y_test))

Бэггинг с базовым алгоритмом "Дерево решений": 0.9883333333333333


In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf5 = BaggingClassifier(base_estimator=LogisticRegression(C=0.615848211066026, penalty='l1', solver='liblinear'))

modelClf5.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Логистическая регрессия с подбором гиперпараметров":',modelClf5.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Логистическая регрессия с подбором гиперпараметров": 0.94


#### Подбор параметров для бэггинга

In [54]:
parameters = {
              'n_estimators':[2,3,4,5,6,8,9,10,20,15],
              'max_samples':[1.0,2.0,3.0],
              #'bootstrap':['True','False'],
              #'bootstrap_features':['True','False'],
              #'oob_score':['True','False']
             }
bgc1 = BaggingClassifier(base_estimator=LogisticRegression())
grid4_ = GridSearchCV(bgc1, parameters)

In [55]:
grid4_.fit(X_train_cls, y_train_cls)
print(grid4_.best_params_)
print(grid4_.best_estimator_)

{'max_samples': 1.0, 'n_estimators': 2}
BaggingClassifier(base_estimator=LogisticRegression(), n_estimators=2)


In [56]:
bgc2 = BaggingClassifier(base_estimator=LogisticRegression(), max_samples=1.0, n_estimators=2)
bgc2.fit(X_train_cls, y_train_cls)
print(bgc2.score(X_test_cls, y_test_cls))

0.945


In [57]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf6 = BaggingClassifier(base_estimator=LogisticRegression(C=0.615848211066026, penalty='l1', solver='liblinear'),max_samples=1.0, n_estimators=5)

modelClf6.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Логистическая регрессия":',modelClf6.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Логистическая регрессия": 0.94


In [58]:
bgc3 = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth = 1, min_samples_leaf = 1, min_samples_split = 2), max_samples=1.0, n_estimators=5)
bgc3.fit(X_train_cls, y_train_cls)
print(bgc3.score(X_test_cls, y_test_cls))

0.87


## Адаптивный бустинг

### Документация 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier

### Адаптивный бустинг без подбора гиперпараметров для него и базовых алгоритмов

In [59]:
from sklearn.ensemble import AdaBoostClassifier

In [60]:
modelClf7 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
modelClf7.fit(X_train_cls, y_train_cls)
print(modelClf7.score(X_test_cls, y_test_cls))

0.9566666666666667


#### Подбор гиперпараметров для бустинга

In [62]:
parameters = {
              'n_estimators':[50,60,70],
              'algorithm': ['SAMME', 'SAMME.R'],
             }
abc1 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
grid5_ = GridSearchCV(abc1, parameters)

In [63]:
grid5_.fit(X_train_cls, y_train_cls)
print(grid5_.best_params_)
print(grid5_.best_estimator_)

{'algorithm': 'SAMME.R', 'n_estimators': 50}
AdaBoostClassifier(base_estimator=DecisionTreeClassifier())


In [64]:
abc2 = AdaBoostClassifier(algorithm='SAMME', base_estimator=DecisionTreeClassifier(),
                   n_estimators=70)
abc2.fit(X_train_cls, y_train_cls)
print(abc2.score(X_test_cls, y_test_cls))

0.9883333333333333


## Градиентный бустинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [65]:
from sklearn.ensemble import GradientBoostingClassifier

In [66]:
modelClf8 = GradientBoostingClassifier()
modelClf8.fit(X_train_cls, y_train_cls)
print(modelClf8.score(X_test_cls, y_test_cls))

0.99


#### Подбор гиперпараметоров для градиентного бустинга

In [67]:
parameters = {
              'n_estimators':[100,150,200,300],
              'max_depth':[2,3,4,5],
              'criterion':['friedman_mse', 'squared_error'],
             }
gbc1 = GradientBoostingClassifier()
grid6_ = GridSearchCV(gbc1, parameters)

In [68]:
grid6_.fit(X_train_cls, y_train_cls)
print(grid6_.best_params_)
print(grid6_.best_estimator_)

{'criterion': 'friedman_mse', 'max_depth': 2, 'n_estimators': 150}
GradientBoostingClassifier(max_depth=2, n_estimators=150)


In [69]:
gbc2 = GradientBoostingClassifier(max_depth=2,criterion='friedman_mse', n_estimators = 100)
gbc2.fit(X_train_cls, y_train_cls)
print(gbc2.score(X_test_cls, y_test_cls))

0.9916666666666667


### Ансамбль методов для регрессии

### Стекинг без подбора гиперпараметров для него и базовых алгоритмов

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html#sklearn.ensemble.StackingRegressor

In [109]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [136]:
student_df = pd.read_csv('../data/student_math_preprocessed.csv')

student_df["useAlc"].unique()
x = credit_df.iloc[:,:-1].values
y = credit_df.iloc[:,4].values
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(x, y,  test_size = 0.2, random_state=0)

In [137]:
estimators = [
    ('dt', DecisionTreeRegressor(random_state=42)),
     ('lr', linear_model.LinearRegression())]
modelClf10 = StackingRegressor(estimators=estimators)

modelClf10.fit(X_train_reg, y_train_reg)
print(modelClf10.score(X_test_reg, y_test_reg))

0.8171296739083087


### Бэггинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html#sklearn.ensemble.BaggingRegressor

In [141]:
from sklearn.ensemble import BaggingRegressor
modelClf12 = BaggingRegressor()
modelClf12.fit(X_train_reg, y_train_reg)
print(modelClf12.score(X_test_reg, y_test_reg))

0.8912298443732795


#### Подбор гиперпараметров для бэггинга

In [142]:
parameters = {
              'n_estimators':[2,3,4,5,6,8,9,10,20,15],
              'max_samples':[1.0,2.0,3.0],
              #'bootstrap':['True','False'],
              #'bootstrap_features':['True','False'],
              #'oob_score':['True','False']
             }
bgr2 = BaggingRegressor(base_estimator=LinearRegression())
grid8_ = GridSearchCV(bgr2, parameters)

In [143]:
grid8_.fit(X_train_reg, y_train_reg)
print(grid8_.best_params_)
print(grid8_.best_estimator_)

{'max_samples': 1.0, 'n_estimators': 2}
BaggingRegressor(base_estimator=LinearRegression(), n_estimators=2)


In [145]:
modelClf14 = BaggingRegressor(base_estimator = DecisionTreeRegressor(),max_samples=1.0, n_estimators=20)
modelClf14.fit(X_train_reg, y_train_reg)
print(modelClf14.score(X_test_reg, y_test_reg))

0.9036462722624867


### Бустинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor

In [146]:
from sklearn.ensemble import AdaBoostRegressor
modelClf15 = AdaBoostRegressor()
modelClf15.fit(X_train_reg,y_train_reg)
print(modelClf15.score(X_test_reg, y_test_reg))

0.6983444356160815


#### Градиентный бустинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor

In [150]:
from sklearn.ensemble import GradientBoostingRegressor
modelClf17 = GradientBoostingRegressor()
modelClf17.fit(X_train_reg, y_train_reg)
print(modelClf17.score(X_test_reg, y_test_reg))

0.8115586416212051


## Catboost

#### Открытый код 
https://github.com/catboost

In [103]:
!pip install catboost

In [104]:
from catboost import CatBoostClassifier
model2 = CatBoostClassifier()
model2.fit(X_train, y_train)

print(model2.score(X_test_cls, y_test_cls));

Learning rate set to 0.011883
0:	learn: 0.6717506	total: 156ms	remaining: 2m 35s
1:	learn: 0.6494350	total: 158ms	remaining: 1m 18s
2:	learn: 0.6287068	total: 159ms	remaining: 53s
3:	learn: 0.6076965	total: 161ms	remaining: 40.2s
4:	learn: 0.5872029	total: 164ms	remaining: 32.6s
5:	learn: 0.5680216	total: 167ms	remaining: 27.6s
6:	learn: 0.5516502	total: 168ms	remaining: 23.9s
7:	learn: 0.5337527	total: 170ms	remaining: 21.1s
8:	learn: 0.5160340	total: 172ms	remaining: 18.9s
9:	learn: 0.5023101	total: 175ms	remaining: 17.3s
10:	learn: 0.4848295	total: 178ms	remaining: 16s
11:	learn: 0.4708754	total: 180ms	remaining: 14.9s
12:	learn: 0.4563068	total: 184ms	remaining: 13.9s
13:	learn: 0.4441661	total: 186ms	remaining: 13.1s
14:	learn: 0.4327678	total: 189ms	remaining: 12.4s
15:	learn: 0.4188321	total: 191ms	remaining: 11.7s
16:	learn: 0.4082354	total: 193ms	remaining: 11.2s
17:	learn: 0.3965961	total: 195ms	remaining: 10.7s
18:	learn: 0.3847365	total: 197ms	remaining: 10.2s
19:	learn: 0.

164:	learn: 0.0580671	total: 483ms	remaining: 2.44s
165:	learn: 0.0577931	total: 486ms	remaining: 2.44s
166:	learn: 0.0574641	total: 488ms	remaining: 2.43s
167:	learn: 0.0571167	total: 490ms	remaining: 2.42s
168:	learn: 0.0567524	total: 493ms	remaining: 2.42s
169:	learn: 0.0564564	total: 496ms	remaining: 2.42s
170:	learn: 0.0563327	total: 499ms	remaining: 2.42s
171:	learn: 0.0561572	total: 502ms	remaining: 2.42s
172:	learn: 0.0558940	total: 505ms	remaining: 2.41s
173:	learn: 0.0555244	total: 508ms	remaining: 2.41s
174:	learn: 0.0552080	total: 514ms	remaining: 2.42s
175:	learn: 0.0548945	total: 516ms	remaining: 2.42s
176:	learn: 0.0546513	total: 519ms	remaining: 2.41s
177:	learn: 0.0545275	total: 523ms	remaining: 2.42s
178:	learn: 0.0543557	total: 526ms	remaining: 2.41s
179:	learn: 0.0542257	total: 529ms	remaining: 2.41s
180:	learn: 0.0539776	total: 532ms	remaining: 2.41s
181:	learn: 0.0536610	total: 535ms	remaining: 2.4s
182:	learn: 0.0533766	total: 537ms	remaining: 2.4s
183:	learn: 0.

343:	learn: 0.0315332	total: 990ms	remaining: 1.89s
344:	learn: 0.0314441	total: 992ms	remaining: 1.88s
345:	learn: 0.0313573	total: 995ms	remaining: 1.88s
346:	learn: 0.0312837	total: 997ms	remaining: 1.88s
347:	learn: 0.0311565	total: 999ms	remaining: 1.87s
348:	learn: 0.0311151	total: 1s	remaining: 1.87s
349:	learn: 0.0310854	total: 1s	remaining: 1.86s
350:	learn: 0.0309790	total: 1.01s	remaining: 1.86s
351:	learn: 0.0308174	total: 1.01s	remaining: 1.86s
352:	learn: 0.0307511	total: 1.01s	remaining: 1.85s
353:	learn: 0.0306849	total: 1.01s	remaining: 1.85s
354:	learn: 0.0306747	total: 1.01s	remaining: 1.84s
355:	learn: 0.0306480	total: 1.02s	remaining: 1.84s
356:	learn: 0.0305973	total: 1.02s	remaining: 1.84s
357:	learn: 0.0305580	total: 1.02s	remaining: 1.83s
358:	learn: 0.0304655	total: 1.02s	remaining: 1.83s
359:	learn: 0.0302835	total: 1.02s	remaining: 1.82s
360:	learn: 0.0301924	total: 1.03s	remaining: 1.82s
361:	learn: 0.0300629	total: 1.03s	remaining: 1.82s
362:	learn: 0.0299

519:	learn: 0.0200508	total: 1.37s	remaining: 1.26s
520:	learn: 0.0200160	total: 1.37s	remaining: 1.26s
521:	learn: 0.0199617	total: 1.37s	remaining: 1.26s
522:	learn: 0.0199112	total: 1.38s	remaining: 1.25s
523:	learn: 0.0197871	total: 1.38s	remaining: 1.25s
524:	learn: 0.0197237	total: 1.38s	remaining: 1.25s
525:	learn: 0.0196708	total: 1.38s	remaining: 1.25s
526:	learn: 0.0196177	total: 1.38s	remaining: 1.24s
527:	learn: 0.0195816	total: 1.39s	remaining: 1.24s
528:	learn: 0.0194886	total: 1.39s	remaining: 1.24s
529:	learn: 0.0194488	total: 1.39s	remaining: 1.23s
530:	learn: 0.0194235	total: 1.39s	remaining: 1.23s
531:	learn: 0.0194143	total: 1.4s	remaining: 1.23s
532:	learn: 0.0193927	total: 1.4s	remaining: 1.23s
533:	learn: 0.0193539	total: 1.4s	remaining: 1.22s
534:	learn: 0.0192835	total: 1.4s	remaining: 1.22s
535:	learn: 0.0192634	total: 1.4s	remaining: 1.22s
536:	learn: 0.0191994	total: 1.41s	remaining: 1.21s
537:	learn: 0.0191299	total: 1.41s	remaining: 1.21s
538:	learn: 0.019

707:	learn: 0.0132788	total: 1.83s	remaining: 754ms
708:	learn: 0.0132279	total: 1.83s	remaining: 752ms
709:	learn: 0.0131837	total: 1.83s	remaining: 749ms
710:	learn: 0.0131728	total: 1.83s	remaining: 746ms
711:	learn: 0.0131479	total: 1.84s	remaining: 744ms
712:	learn: 0.0131447	total: 1.84s	remaining: 741ms
713:	learn: 0.0131201	total: 1.84s	remaining: 739ms
714:	learn: 0.0131148	total: 1.85s	remaining: 736ms
715:	learn: 0.0130910	total: 1.85s	remaining: 734ms
716:	learn: 0.0130728	total: 1.85s	remaining: 731ms
717:	learn: 0.0130698	total: 1.85s	remaining: 728ms
718:	learn: 0.0130427	total: 1.86s	remaining: 725ms
719:	learn: 0.0130181	total: 1.86s	remaining: 723ms
720:	learn: 0.0130154	total: 1.86s	remaining: 720ms
721:	learn: 0.0129566	total: 1.86s	remaining: 717ms
722:	learn: 0.0129132	total: 1.86s	remaining: 714ms
723:	learn: 0.0129008	total: 1.87s	remaining: 712ms
724:	learn: 0.0128845	total: 1.87s	remaining: 709ms
725:	learn: 0.0128816	total: 1.87s	remaining: 706ms
726:	learn: 

871:	learn: 0.0104181	total: 2.17s	remaining: 318ms
872:	learn: 0.0104157	total: 2.17s	remaining: 316ms
873:	learn: 0.0104136	total: 2.17s	remaining: 313ms
874:	learn: 0.0103838	total: 2.17s	remaining: 311ms
875:	learn: 0.0103803	total: 2.18s	remaining: 308ms
876:	learn: 0.0103560	total: 2.18s	remaining: 306ms
877:	learn: 0.0103529	total: 2.18s	remaining: 303ms
878:	learn: 0.0103517	total: 2.18s	remaining: 301ms
879:	learn: 0.0103330	total: 2.19s	remaining: 298ms
880:	learn: 0.0103319	total: 2.19s	remaining: 296ms
881:	learn: 0.0103059	total: 2.19s	remaining: 293ms
882:	learn: 0.0102875	total: 2.19s	remaining: 291ms
883:	learn: 0.0102619	total: 2.2s	remaining: 288ms
884:	learn: 0.0102601	total: 2.2s	remaining: 286ms
885:	learn: 0.0102350	total: 2.2s	remaining: 283ms
886:	learn: 0.0102329	total: 2.2s	remaining: 281ms
887:	learn: 0.0102148	total: 2.21s	remaining: 278ms
888:	learn: 0.0102139	total: 2.21s	remaining: 276ms
889:	learn: 0.0101961	total: 2.21s	remaining: 273ms
890:	learn: 0.01

In [105]:
model2.evals_result_

{'learn': {'Logloss': [0.6717506481218568,
   0.6494350148606232,
   0.6287067532566034,
   0.6076965431976762,
   0.5872028645504007,
   0.5680215626944366,
   0.5516502107456913,
   0.5337527136164387,
   0.5160339781767825,
   0.5023100797670947,
   0.48482949088834915,
   0.47087543255615505,
   0.4563068400857785,
   0.44416609210206853,
   0.4327678298664332,
   0.41883213852206214,
   0.40823543415209523,
   0.3965960593844791,
   0.3847365199282243,
   0.3759054638013382,
   0.363807474385011,
   0.3524861841561885,
   0.3430419251930739,
   0.33343000662011085,
   0.3235865351807329,
   0.3163682934251443,
   0.30680858175841585,
   0.2984218740983784,
   0.2914783706089899,
   0.28294646010368146,
   0.27471235407277694,
   0.2683290619037795,
   0.26242989568345104,
   0.25599598602985774,
   0.25017215116329505,
   0.24386786094032703,
   0.23840226854865007,
   0.23265634928333306,
   0.22770309160245514,
   0.2222709925625268,
   0.2165498970957765,
   0.21134923693957974

In [106]:
model2.best_score_

{'learn': {'Logloss': 0.008829643492538244}}

In [154]:
from catboost import CatBoostRegressor
model21 = CatBoostRegressor()
model21.fit(X_train_reg, y_train_reg)

print(model21.score(X_test_reg, y_test_reg));

Learning rate set to 0.044086
0:	learn: 0.3419148	total: 1.97ms	remaining: 1.97s
1:	learn: 0.3314160	total: 4.03ms	remaining: 2.01s
2:	learn: 0.3227025	total: 6.11ms	remaining: 2.03s
3:	learn: 0.3135060	total: 7.89ms	remaining: 1.96s
4:	learn: 0.3048857	total: 9.75ms	remaining: 1.94s
5:	learn: 0.2960005	total: 11.4ms	remaining: 1.89s
6:	learn: 0.2870758	total: 13.1ms	remaining: 1.86s
7:	learn: 0.2777447	total: 15.3ms	remaining: 1.89s
8:	learn: 0.2701049	total: 17.2ms	remaining: 1.9s
9:	learn: 0.2630847	total: 19.3ms	remaining: 1.91s
10:	learn: 0.2552304	total: 39.7ms	remaining: 3.57s
11:	learn: 0.2483470	total: 42.7ms	remaining: 3.51s
12:	learn: 0.2416798	total: 47.7ms	remaining: 3.62s
13:	learn: 0.2360635	total: 49.8ms	remaining: 3.51s
14:	learn: 0.2310218	total: 57.1ms	remaining: 3.75s
15:	learn: 0.2247072	total: 60.7ms	remaining: 3.73s
16:	learn: 0.2187874	total: 62.9ms	remaining: 3.64s
17:	learn: 0.2140223	total: 65.7ms	remaining: 3.58s
18:	learn: 0.2090488	total: 67.9ms	remaining:

194:	learn: 0.0814978	total: 326ms	remaining: 1.34s
195:	learn: 0.0814609	total: 328ms	remaining: 1.34s
196:	learn: 0.0813920	total: 330ms	remaining: 1.34s
197:	learn: 0.0813731	total: 332ms	remaining: 1.34s
198:	learn: 0.0813416	total: 333ms	remaining: 1.34s
199:	learn: 0.0812841	total: 335ms	remaining: 1.34s
200:	learn: 0.0812418	total: 337ms	remaining: 1.34s
201:	learn: 0.0812077	total: 338ms	remaining: 1.33s
202:	learn: 0.0811732	total: 340ms	remaining: 1.33s
203:	learn: 0.0810862	total: 342ms	remaining: 1.33s
204:	learn: 0.0809271	total: 343ms	remaining: 1.33s
205:	learn: 0.0809080	total: 345ms	remaining: 1.33s
206:	learn: 0.0808920	total: 347ms	remaining: 1.33s
207:	learn: 0.0806264	total: 349ms	remaining: 1.33s
208:	learn: 0.0805353	total: 351ms	remaining: 1.33s
209:	learn: 0.0804898	total: 353ms	remaining: 1.33s
210:	learn: 0.0801247	total: 354ms	remaining: 1.32s
211:	learn: 0.0800880	total: 355ms	remaining: 1.32s
212:	learn: 0.0800317	total: 357ms	remaining: 1.32s
213:	learn: 

409:	learn: 0.0624121	total: 656ms	remaining: 944ms
410:	learn: 0.0623756	total: 658ms	remaining: 944ms
411:	learn: 0.0622274	total: 660ms	remaining: 942ms
412:	learn: 0.0622051	total: 662ms	remaining: 941ms
413:	learn: 0.0621872	total: 663ms	remaining: 939ms
414:	learn: 0.0620990	total: 665ms	remaining: 937ms
415:	learn: 0.0620674	total: 666ms	remaining: 936ms
416:	learn: 0.0619619	total: 668ms	remaining: 934ms
417:	learn: 0.0617611	total: 670ms	remaining: 933ms
418:	learn: 0.0617388	total: 672ms	remaining: 931ms
419:	learn: 0.0616847	total: 673ms	remaining: 930ms
420:	learn: 0.0615617	total: 676ms	remaining: 930ms
421:	learn: 0.0615462	total: 678ms	remaining: 928ms
422:	learn: 0.0615262	total: 679ms	remaining: 927ms
423:	learn: 0.0614700	total: 681ms	remaining: 925ms
424:	learn: 0.0613862	total: 682ms	remaining: 923ms
425:	learn: 0.0611982	total: 684ms	remaining: 921ms
426:	learn: 0.0611771	total: 685ms	remaining: 919ms
427:	learn: 0.0611561	total: 686ms	remaining: 917ms
428:	learn: 

614:	learn: 0.0517210	total: 988ms	remaining: 618ms
615:	learn: 0.0516598	total: 990ms	remaining: 617ms
616:	learn: 0.0515164	total: 991ms	remaining: 615ms
617:	learn: 0.0514476	total: 993ms	remaining: 614ms
618:	learn: 0.0513941	total: 995ms	remaining: 612ms
619:	learn: 0.0513589	total: 996ms	remaining: 611ms
620:	learn: 0.0512049	total: 998ms	remaining: 609ms
621:	learn: 0.0511929	total: 1s	remaining: 608ms
622:	learn: 0.0511713	total: 1s	remaining: 606ms
623:	learn: 0.0511463	total: 1s	remaining: 605ms
624:	learn: 0.0510028	total: 1s	remaining: 604ms
625:	learn: 0.0509801	total: 1.01s	remaining: 602ms
626:	learn: 0.0509614	total: 1.01s	remaining: 600ms
627:	learn: 0.0509021	total: 1.01s	remaining: 599ms
628:	learn: 0.0508583	total: 1.01s	remaining: 597ms
629:	learn: 0.0508176	total: 1.01s	remaining: 595ms
630:	learn: 0.0507683	total: 1.01s	remaining: 594ms
631:	learn: 0.0507422	total: 1.02s	remaining: 592ms
632:	learn: 0.0507090	total: 1.02s	remaining: 590ms
633:	learn: 0.0506464	to

786:	learn: 0.0434878	total: 1.31s	remaining: 355ms
787:	learn: 0.0434857	total: 1.31s	remaining: 354ms
788:	learn: 0.0434512	total: 1.32s	remaining: 352ms
789:	learn: 0.0434322	total: 1.32s	remaining: 351ms
790:	learn: 0.0434200	total: 1.32s	remaining: 349ms
791:	learn: 0.0433869	total: 1.32s	remaining: 348ms
792:	learn: 0.0433688	total: 1.33s	remaining: 346ms
793:	learn: 0.0433293	total: 1.33s	remaining: 345ms
794:	learn: 0.0431809	total: 1.33s	remaining: 343ms
795:	learn: 0.0431632	total: 1.33s	remaining: 342ms
796:	learn: 0.0430269	total: 1.33s	remaining: 340ms
797:	learn: 0.0429608	total: 1.33s	remaining: 338ms
798:	learn: 0.0429347	total: 1.34s	remaining: 337ms
799:	learn: 0.0428972	total: 1.34s	remaining: 335ms
800:	learn: 0.0428613	total: 1.34s	remaining: 333ms
801:	learn: 0.0428204	total: 1.34s	remaining: 332ms
802:	learn: 0.0427953	total: 1.34s	remaining: 330ms
803:	learn: 0.0427272	total: 1.35s	remaining: 328ms
804:	learn: 0.0426958	total: 1.35s	remaining: 327ms
805:	learn: 

978:	learn: 0.0361359	total: 1.64s	remaining: 35.2ms
979:	learn: 0.0360727	total: 1.64s	remaining: 33.5ms
980:	learn: 0.0360589	total: 1.64s	remaining: 31.8ms
981:	learn: 0.0360478	total: 1.65s	remaining: 30.2ms
982:	learn: 0.0360219	total: 1.65s	remaining: 28.5ms
983:	learn: 0.0359927	total: 1.65s	remaining: 26.8ms
984:	learn: 0.0359209	total: 1.65s	remaining: 25.1ms
985:	learn: 0.0358635	total: 1.65s	remaining: 23.5ms
986:	learn: 0.0358228	total: 1.65s	remaining: 21.8ms
987:	learn: 0.0358140	total: 1.65s	remaining: 20.1ms
988:	learn: 0.0357474	total: 1.66s	remaining: 18.4ms
989:	learn: 0.0357402	total: 1.66s	remaining: 16.7ms
990:	learn: 0.0356887	total: 1.66s	remaining: 15.1ms
991:	learn: 0.0356655	total: 1.66s	remaining: 13.4ms
992:	learn: 0.0356342	total: 1.66s	remaining: 11.7ms
993:	learn: 0.0356017	total: 1.66s	remaining: 10ms
994:	learn: 0.0355805	total: 1.67s	remaining: 8.37ms
995:	learn: 0.0355669	total: 1.67s	remaining: 6.7ms
996:	learn: 0.0355289	total: 1.67s	remaining: 5.0

In [155]:
model21.best_score_

{'learn': {'RMSE': 0.03543607433670662}}